In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
from pathlib import Path
from sklearn.preprocessing import StandardScaler

Highlighting some changes made in the dataset. To save time I did the changes in Excel. The data was imported after making these changes.  
1. I started with the results.csv dataset. I renamed the status column to "finish_status" and changed the column to only 1s and 2s. In the original data, we have 1 for race finish and and whole series of codes from 2 to 139 which represent different outcomes but are all essentially "Did not Finish (DNF)" for different reasons. I changed all of these to 2 since I only care about race finishes. 
2. I changed the ending position to 0 for rows where "finish_status = 2"
3. I added the circuit_Id column
4. I only filtered for races between 2009 to 2018. The reason for this is because essentially every driver that was on the grid in 2019 only started racing from 2009. In the future, I might switch this to only include 2014 to 2018 and see how the model performs. This is because the existing engine (V6 Engines) regulations came into effect from 2014. The engines in 2009 to 2013 were slightly different (V8 Engines)
5. Added a train_test column. Races from 2009 to 2018 are marked as 1 while 2019 data is marked as 2. 

In [2]:
df = pd.read_csv(Path('race_results.csv'))
df.head()

,ending_position,raceId,curcuit_id,driverId,constructorId,starting_position,finish_status,train_test,resultId
0,1,337,3,4,6,3,1,1,20323
1,2,337,3,13,6,2,1,1,20324
2,3,337,3,1,1,4,1,1,20325
3,4,337,3,20,9,1,1,1,20326
4,5,337,3,3,131,5,1,1,20327


Create a new dataframe to only filter for rows where driver status = 1 aka. the drivers were able to finish the race.

In [3]:
df1 = df.loc[(df["finish_status"] == 1)]

In [4]:
# Creating our train and test data sets using the train_test column
df_train = df1.loc[df["train_test"] == 1]
df_test = df1.loc[df["train_test"] == 2]

In [5]:
X_train = df_train[['curcuit_id','driverId','constructorId', 'starting_position']]
y_train = df_train['ending_position']

In [6]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [7]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', multi_class='multinomial')
classifier

LogisticRegression(multi_class='multinomial')

In [8]:
classifier.fit(X_train_scaled, y_train)

/Users/shahkibria/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(multi_class='multinomial')

In [9]:
X_test = df_test[['curcuit_id','driverId','constructorId', 'starting_position']]
y_test = df_test['ending_position']

In [10]:
scaler1 = StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [11]:
y_pred = classifier.predict(X_test_scaled)
Z = pd.DataFrame({"Prediction": y_pred, "Actual": y_test})
Z

,Prediction,Actual
3877,2,1
3878,1,2
3879,5,3
3880,1,4
3881,5,5
...,...,...
4291,17,15
4292,14,16
4293,12,17
4294,10,18


In [12]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.15555555555555556


In [13]:
# Merging the Z dataframe and df_test datframe 
df_test1 = pd.merge(df_test, Z, left_index=True, right_index=True, how='outer')

Presenting our results for predicting race winners. We got 15 out of 21 racewinners correct

In [14]:
df_test1.loc[(df_test1["Prediction"] == 1) & (df_test1["ending_position"] == 1)]

,ending_position,raceId,curcuit_id,driverId,constructorId,starting_position,finish_status,train_test,resultId,Prediction,Actual
3917,1,1012,17,1,131,2,1,2,24243,1,1
3937,1,1013,73,822,131,1,1,2,24263,1,1
3957,1,1014,4,1,131,2,1,2,24283,1,1
3977,1,1015,6,1,131,1,1,2,24306,1,1
3997,1,1016,7,1,131,2,1,2,24326,1,1
4017,1,1017,34,1,131,1,1,2,24346,1,1
4057,1,1019,9,1,131,2,1,2,24386,1,1
4117,1,1022,13,844,6,1,1,2,24446,1,1
4137,1,1023,14,844,6,1,1,2,24466,1,1
4157,1,1024,15,20,6,3,1,2,24486,1,1


Presenting our results for predicting race winners. We got 7 out of 21 2nd place finish correct

In [15]:
df_test1.loc[(df_test1["Prediction"] == 2) & (df_test1["ending_position"] == 2)]

,ending_position,raceId,curcuit_id,driverId,constructorId,starting_position,finish_status,train_test,resultId,Prediction,Actual
3898,2,1011,3,822,131,4,1,2,24224,2,2
4018,2,1017,34,822,131,2,1,2,24347,2,2
4118,2,1022,13,1,131,3,1,2,24447,2,2
4138,2,1023,14,822,131,3,1,2,24467,2,2
4178,2,1025,71,822,131,4,1,2,24507,2,2
4238,2,1028,69,1,131,5,1,2,24567,2,2
4278,2,1030,24,830,9,2,1,2,24607,2,2
